# Results of Global search 
## Synthetic 3 Layer case

In [1]:
%matplotlib widget

In [2]:
# Import libraries
#import empymod
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import mu_0
import pandas as pd
import pygimli as pg

import sys
sys.path.insert(1, '../../src')

from PlotModel import PlotModelCond_3lay, PlotErrorSpace, PlotData, rmse_a
from EM1D import EMforward3lay

### Show Model

In [ ]:
# Import the conductivities and thicknesses used to create the LU table

conds = np.load('../data/conds.npy')
thick = np.load('../data/thicks.npy')

model_true = np.load('../data/B1_w/model_synth_w.npy')
model_est_FS = np.load('../data/B1_w/model_est_FS_w.npy')
model_est_FS_Q = np.load('../data/B1_Q_w/model_est_FS_B1_Q_w.npy')
model_est_FS_IP =  np.load('../data/B1_IP_w/model_est_FS_B1_IP_w.npy')

data_true =  np.load('../data/B1_w/data_synth_w.npy')

# Load with noise
model_est_FS_noise2 = np.load('../data/B1_noise2_w/model_est_FS_B1_noise2_w.npy')
model_est_FS_noise5 = np.load('../data/B1_noise5_w/model_est_FS_B1_noise5_w.npy')
model_est_FS_noise10 = np.load('../data/B1_noise10_w/model_est_FS_B1_noise10_w.npy')

data_true_noise2 = np.load('../data/B1_noise2_w/data_synth_noise2_w.npy')
data_true_noise5 = np.load('../data/B1_noise5_w/data_synth_noise5_w.npy')
data_true_noise10 =  np.load('../data/B1_noise10_w/data_synth_noise10_w.npy')

npos = len(model_true)
nlayer = 3

depths_true = np.zeros((npos, nlayer))
sigmas_true = np.zeros((npos, nlayer))

depths_true[:,1] = model_true[:,3]
depths_true[:,2] = model_true[:,3] + model_true[:,4]
sigmas_true[:,:] = model_true[:,:3]

In [ ]:
model_true_grid = PlotModelCond_3lay(sigmas_true, depths_true[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Stitched section 1D true models')
plt.text(0.1,-0.5,'a)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
depths_est_FS = np.zeros_like(depths_true)
sigmas_est_FS = np.zeros_like(sigmas_true)

depths_est_FS[:,1] = model_est_FS[:,3]
depths_est_FS[:,2] = model_est_FS[:,3] + model_est_FS[:,4]
sigmas_est_FS[:,:] = model_est_FS[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid = PlotModelCond_3lay(sigmas_est_FS, depths_est_FS[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [Q +IP]')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'b)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q + IP]')
plt.text(0.1,-0.5,'e)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
depths_est_FS_Q = np.zeros_like(depths_true)
sigmas_est_FS_Q = np.zeros_like(sigmas_true)

depths_est_FS_Q[:,1] = model_est_FS_Q[:,3]
depths_est_FS_Q[:,2] = model_est_FS_Q[:,3] + model_est_FS_Q[:,4]
sigmas_est_FS_Q[:,:] = model_est_FS_Q[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS_Q)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS_Q[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_Q= PlotModelCond_3lay(sigmas_est_FS_Q, depths_est_FS_Q[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [Q]')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'c)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_Q)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q]')
plt.text(0.1,-0.5,'f)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
depths_est_FS_IP = np.zeros_like(depths_true)
sigmas_est_FS_IP = np.zeros_like(sigmas_true)

depths_est_FS_IP[:,1] = model_est_FS_IP[:,3]
depths_est_FS_IP[:,2] = model_est_FS_IP[:,3] + model_est_FS_IP[:,4]
sigmas_est_FS_IP[:,:] = model_est_FS_IP[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS_IP)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS_IP[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_IP = PlotModelCond_3lay(sigmas_est_FS_IP, depths_est_FS_IP[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [IP]')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'d)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_IP)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [IP]')
plt.text(0.1,-0.5,'g)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

### Test minimization scheme

In [ ]:
# Create forward modeller class

class EMforward_3lay(pg.Modelling):
    def __init__(self):
        """Initialize the model."""
        super().__init__()        
    def response(self, x):
        sig1 = x[0]
        sig2 = x[1]
        sig3 = x[2]
        thk1 = x[3]
        thk2 = x[4]
        if (thk1+thk2) >= 10:
            thk1 = 2
            thk2 = 3
        Z = EMforward3lay(sig1, sig2, sig3, thk1, thk2, height=0.47)                           
        return Z               
    def createStartModel(self, dataVals):
        thk_ini = [2,3]
        sig_ini = [1/20,1/20, 1/20]
        x0 = sig_ini + thk_ini
        return np.array(x0)

In [ ]:
# Initialize the forward modelling class
EMf_3lay = EMforward_3lay()

In [ ]:
# Relative error array
error = 1e-3 # introduce here the error you want to test
relativeError = np.ones_like(data_true[0]) * error

In [ ]:
# Define inversion transformations

transData = pg.trans.TransLog() # log transformation for data
transModel = pg.trans.TransLog()  # log-transform ensures thk and res >0

# Create inversion
invEM = pg.Inversion()
invEM.setForwardOperator(EMf_3lay)

invEM.transModel = transModel
invEM.transData = transData

In [ ]:
# Start inversion

model_est = np.zeros_like(model_true)
#chi2hist = [] # empty array to store the chi2 history
#modelhist = [] # empty array to store the model history

# Perform inversion for each 1D model per position in stitched section
for pos in range(npos):
    dataE = data_true[pos].copy()
    dataE *= np.random.randn(len(dataE)) * relativeError + 1.0
    model_est_pos = invEM.run(dataE, relativeError, verbose=False, lam=10, maxIter=40)
    #chi2hist.append(invEM.chi2History)
    #modelhist.append(invEM.modelHistory)
    model_est[pos] = model_est_pos

In [ ]:
depths_est = np.zeros_like(depths_true)
sigmas_est = np.zeros_like(sigmas_true)
depths_est[:,1] = model_est[:,3]
depths_est[:,2] = model_est[:,3] + model_est[:,4]
sigmas_est[:,:] = model_est[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_Opt = PlotModelCond_3lay(sigmas_est, depths_est[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')

plt.title('Optimization: [Q + IP]')
plt.text(0.1,-0.5,'h)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_Opt)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q + IP]')
plt.text(0.1,-0.5,'k)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
# Create forward modeller class

class EMforward_3lay_Q(pg.Modelling):
    def __init__(self):
        """Initialize the model."""
        super().__init__()        
    def response(self, x):
        sig1 = x[0]
        sig2 = x[1]
        sig3 = x[2]
        thk1 = x[3]
        thk2 = x[4]
        if (thk1+thk2) >= 10:
            thk1 = 2
            thk2 = 3
        Z = EMforward3lay(sig1, sig2, sig3, thk1, thk2, height=0.47)                           
        return Z[:9]
    def createStartModel(self, dataVals):
        thk_ini = [2,3]
        sig_ini = [1/20,1/20, 1/20]
        x0 = sig_ini + thk_ini
        return np.array(x0)

In [ ]:
# Initialize the forward modelling class
EMf_3lay_Q = EMforward_3lay_Q()

# Relative error array
error = 1e-3 # introduce here the error you want to test
relativeError = np.ones_like(data_true[0,:9]) * error

# Define inversion transformations
transData = pg.trans.TransLog() # log transformation for data
transModel = pg.trans.TransLog()  # log-transform ensures thk and res >0

# Create inversion
invEM = pg.Inversion()
invEM.setForwardOperator(EMf_3lay_Q)

invEM.transModel = transModel
invEM.transData = transData

# Start inversion
model_est_Q = np.zeros_like(model_true)
#chi2hist = [] # empty array to store the chi2 history
#modelhist = [] # empty array to store the model history

# Perform inversion for each 1D model per position in stitched section
for pos in range(npos):
    dataE = data_true[pos,:9].copy()
    dataE *= np.random.randn(len(dataE)) * relativeError + 1.0
    model_est_pos = invEM.run(dataE, relativeError, verbose=False, lam=10, maxIter=40)
    #chi2hist.append(invEM.chi2History)
    #modelhist.append(invEM.modelHistory)
    model_est_Q[pos] = model_est_pos

In [ ]:
depths_est_Q = np.zeros_like(depths_true)
sigmas_est_Q = np.zeros_like(sigmas_true)
depths_est_Q[:,1] = model_est_Q[:,3]
depths_est_Q[:,2] = model_est_Q[:,3] + model_est_Q[:,4]
sigmas_est_Q[:,:] = model_est_Q[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_Q)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_Q[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_Opt_Q = PlotModelCond_3lay(sigmas_est_Q, depths_est_Q[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'i)', color='k',fontsize=15)
plt.title('Optimization: [Q]')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_Opt_Q)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q]')
plt.text(0.1,-0.5,'l)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
# Create forward modeller class

class EMforward_3lay_IP(pg.Modelling):
    def __init__(self):
        """Initialize the model."""
        super().__init__()        
    def response(self, x):
        sig1 = x[0]
        sig2 = x[1]
        sig3 = x[2]
        thk1 = x[3]
        thk2 = x[4]
        if (thk1+thk2) >= 10:
            thk1 = 2
            thk2 = 3
        Z = EMforward3lay(sig1, sig2, sig3, thk1, thk2, height=0.47)                           
        return Z[9:]
    def createStartModel(self, dataVals):
        thk_ini = [2,3]
        sig_ini = [1/20,1/20, 1/20]
        x0 = sig_ini + thk_ini
        return np.array(x0)

In [ ]:
# Initialize the forward modelling class
EMf_3lay_IP = EMforward_3lay_IP()

# Relative error array
error = 1e-3 # introduce here the error you want to test
relativeError = np.ones_like(data_true[0,9:]) * error

# Define inversion transformations
transData = pg.trans.TransLog() # log transformation for data
transModel = pg.trans.TransLog()  # log-transform ensures thk and res >0

# Create inversion
invEM = pg.Inversion()
invEM.setForwardOperator(EMf_3lay_IP)

invEM.transModel = transModel
invEM.transData = transData

# Start inversion
model_est_IP = np.zeros_like(model_true)
#chi2hist = [] # empty array to store the chi2 history
#modelhist = [] # empty array to store the model history

# Perform inversion for each 1D model per position in stitched section
for pos in range(npos):
    dataE = data_true[pos,9:].copy()
    dataE *= np.random.randn(len(dataE)) * relativeError + 1.0
    model_est_pos = invEM.run(dataE, relativeError, verbose=False, lam=10, maxIter=40)
    #chi2hist.append(invEM.chi2History)
    #modelhist.append(invEM.modelHistory)
    model_est_IP[pos] = model_est_pos

In [ ]:
depths_est_IP = np.zeros_like(depths_true)
sigmas_est_IP = np.zeros_like(sigmas_true)
depths_est_IP[:,1] = model_est_IP[:,3]
depths_est_IP[:,2] = model_est_IP[:,3] + model_est_IP[:,4]
sigmas_est_IP[:,:] = model_est_IP[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_IP)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_IP[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_Opt_IP = PlotModelCond_3lay(sigmas_est_IP, depths_est_IP[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'j)', color='k',fontsize=15)
plt.title('Optimization: [IP]')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_Opt_IP)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [IP]')
plt.text(0.1,-0.5,'m)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

### Noise

In [ ]:
depths_est_FS_noise2 = np.zeros_like(depths_true)
sigmas_est_FS_noise2 = np.zeros_like(sigmas_true)

depths_est_FS_noise2[:,1] = model_est_FS_noise2[:,3]
depths_est_FS_noise2[:,2] = model_est_FS_noise2[:,3] + model_est_FS_noise2[:,4]
sigmas_est_FS_noise2[:,:] = model_est_FS_noise2[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS_noise2)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS_noise2[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_noise2 = PlotModelCond_3lay(sigmas_est_FS_noise2, depths_est_FS_noise2[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [Q + IP], noise: 2.5%')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'a)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_noise2)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q + IP], noise: 2.5%')
plt.text(0.1,-0.5,'d)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
depths_est_FS_noise5 = np.zeros_like(depths_true)
sigmas_est_FS_noise5 = np.zeros_like(sigmas_true)

depths_est_FS_noise5[:,1] = model_est_FS_noise5[:,3]
depths_est_FS_noise5[:,2] = model_est_FS_noise5[:,3] + model_est_FS_noise5[:,4]
sigmas_est_FS_noise5[:,:] = model_est_FS_noise5[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS_noise5)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS_noise5[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_noise5 = PlotModelCond_3lay(sigmas_est_FS_noise5, depths_est_FS_noise5[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [Q + IP], noise: 5%')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'b)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_noise5)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q + IP], noise: 5%')
plt.text(0.1,-0.5,'e)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')

In [ ]:
depths_est_FS_noise10 = np.zeros_like(depths_true)
sigmas_est_FS_noise10 = np.zeros_like(sigmas_true)

depths_est_FS_noise10[:,1] = model_est_FS_noise10[:,3]
depths_est_FS_noise10[:,2] = model_est_FS_noise10[:,3] + model_est_FS_noise10[:,4]
sigmas_est_FS_noise10[:,:] = model_est_FS_noise10[:,:3]

RMSE_res = rmse_a(np.log10(np.hstack(sigmas_est_FS_noise10)), np.log10(np.hstack(sigmas_true)))
RMSE_thk = rmse_a(np.hstack(depths_est_FS_noise10[:,1:]), np.hstack(depths_true[:,1:]))

model_est_grid_noise10 = PlotModelCond_3lay(sigmas_est_FS_noise10, depths_est_FS_noise5[:,1:])
xx = np.linspace(0,npos+1,npos+1, endpoint=False)
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,1],depths_true[:,1], depths_true[-2,1])), ':r')
plt.step(np.hstack((xx, xx[-1])), np.hstack((depths_true[0,2],depths_true[:,2], depths_true[-2,2])), ':r')
plt.title('Global search: [Q + IP], noise: 10%')
plt.text(0,9,' $\sigma$ RMSE %.3g' %(RMSE_res*100) + '%' 
          +' \n $h$ RMSE %.3g' %(RMSE_thk*100) +'%', color='w')
plt.text(0.1,-0.5,'c)', color='k',fontsize=15)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

error_est = np.abs((model_true_grid-model_est_grid_noise10)/model_true_grid)*100+0.1
cf = ax.imshow(np.log10(error_est).T, extent=[0,npos,10,0], vmin=0, vmax=2,
          cmap=plt.cm.get_cmap("RdBu_r", 8))
cbar = fig.colorbar(cf, ax=ax, label=r"Relative Error",  shrink=0.5)
cbar.set_ticks([0, 1, 2])
cbar.ax.set_yticklabels([ r"$1\,\%$",
                         r"$10\,\%$", r"$100\,\%$"])
plt.title('Relative Error [Q + IP], noise: 10%')
plt.text(0.1,-0.5,'f)', color='k',fontsize=15)
plt.xlabel('Horizontal distance [m]')
plt.ylabel('Depth [m]')